In [1]:
 # local-ness factor 
frac_close = 0.03
frac_broad= 0.3
smoothing_sigma = 150
fit_close_cutoff = 300


i = 0



for spectrum, filepath in zip(spectrums, filepaths):
    i+=1
    if i > 10:
        break

    fit_broad = lowess(spectrum,f,frac=frac_broad, return_sorted=False)
    fit_close = lowess(spectrum,f,frac=frac_close, return_sorted=False)
    min_spectrum = np.minimum(fit_broad, fit_close)
    min_spectrum2 = np.minimum(fit_broad, spectrum)
    min_spectrum[0:fit_close_cutoff] = fit_close[0:fit_close_cutoff]
    min_spectrum2[0:fit_close_cutoff] = spectrum[0:fit_close_cutoff]
    noise_floor1 = gaussian_filter(min_spectrum, sigma=smoothing_sigma)
    noise_floor2 = gaussian_filter(min_spectrum2, sigma=smoothing_sigma)
    # sum=0
    # for j in range(len(min_spectrum)):
    #     if min_spectrum[j] < min_spectrum2[j]:
    #         sum+=1
    # print(sum)
    
    plt.plot(f, spectrum, label='Original Spectrum')
    # plt.plot(f, min_spectrum, 'black', label='min_spectrum')
    # plt.plot(f, min_spectrum2, label='min_spectrum2')
    # plt.plot(f, fit_broad,'g--',label='Broad LOWESS Fit')
    # plt.plot(f, fit_close,'r--',label='Close LOWESS Fit')
    plt.plot(f, noise_floor1,'black',label='Noise Floor 1')
    plt.plot(f, noise_floor2,'green',label='Noise Floor 2')
    plt.legend()
    plt.title(filepath)
    plt.show()



NameError: name 'spectrums' is not defined

In [ ]:
    print(len(min_spectrum)-300)
    ## Apply Gaussian filter with different sigmas
    smoothed1 = gaussian_filter(min_spectrum[:split_point + transition_width], sigma=sigma_large)
    smoothed2 = gaussian_filter(min_spectrum[split_point:], sigma=sigma_small)
    
    # Blend transition region
    blended_transition = np.linspace(1, 0, transition_width) * smoothed1[-transition_width:] + \
                         np.linspace(0, 1, transition_width) * smoothed2[:transition_width]

    print(len(smoothed1[:-transition_width]), " ", len(smoothed2[transition_width:]), " ", len(blended_transition))

    
    # Combine the filtered parts with smooth transition
    noise_floor = np.concatenate([
        smoothed1[:-transition_width],
        blended_transition,
        smoothed2[transition_width:]
    ])


In [ ]:
# Define a weighted loss function based on the variable choice

# Loss weights
weighted_loss_type = 'exponential'  # Options: 'exponential' or 'square'
weight_min = 300                  # Minimal freq_bin index to weight. Avoids weighting the near zero "peak".

def weighted_mse(y_true, y_pred):
    # Calculate weights based on the weighted_loss_type
    if weighted_loss_type == 'exponential':
        weights = tf.exp(y_true)
    elif weighted_loss_type == 'square':
        weights = tf.square(y_true)
    else:
        raise ValueError("Invalid weighted_loss_type. Choose 'exponential' or 'square'.")

    # Dynamically compute the shape to handle variable batch sizes
    batch_size = tf.shape(weights)[0]
    time_steps = tf.shape(weights)[1]

    # Create a mask for indices below the weight_min
    mask = tf.range(time_steps) < weight_min  # Shape: (time_steps,)
    mask = tf.expand_dims(mask, axis=0)       # Shape: (1, time_steps)
    mask = tf.expand_dims(mask, axis=-1)      # Shape: (1, time_steps, 1)
    mask = tf.tile(mask, [batch_size, 1, 1])  # Shape: (batch_size, time_steps, 1)

    # Replace weights below the cutoff with the minimum weight value
    min_weight = tf.reduce_min(weights[:, weight_min:, :], axis=1, keepdims=True)
    weights = tf.where(mask, tf.broadcast_to(min_weight, tf.shape(weights)), weights)

    # Compute the weighted mean squared error
    loss = tf.reduce_mean(weights * tf.square(y_true - y_pred))
    return loss


In [ ]:
import numpy as np
from pathlib import Path
import statsmodels.api as sm

# Get noise floor files

f = rfftfreq(32768, 1/44100)[0:-1]

"""Take FFT of all raw waveforms in the dataframe and convert to dB SPL"""
# Define constants that allow us to correct for experimental setup
amp_factor = 0.01  # Correct for amplifier gain
# amp_factor = 0.1  # Correct for amplifier gain
mic_factor = 10**(-6)  # Corresponds to the mic level of 1 microvolt
# mic_factor = 1  # Corresponds to the mic level of 1 microvolt
rms_factor = np.sqrt(2)  # Converts to RMS value

# Define window length
n_win = 32768

# First navigate to our directory
directory_path = Path(r"C:\Users\Owner\OneDrive\Desktop\Tube Noise Floors")
# directory_path = Path(r"C:\Users\Owner\OneDrive\Desktop\SOAE Data\Tube Files")

# now loop through all files in that collection
for fp in directory_path.rglob('*'):
    if fp.is_file() == False:  
        continue
    elif "supp" in fp.name:
        noise_floor = np.loadtxt(fp)[:, 1]

    else:
        wf = np.loadtxt(fp)
        # Divide the waveform into windows of length n_win, take magnitude of FFT of each
        mags_list = [
            np.abs(rfft(wf[i * n_win:(i + 1) * n_win])*(2/n_win))
            for i in range(len(wf) // n_win)
        ]

        # Average over all windows
        avg_mags = np.mean(mags_list, axis=0)
        noise_floor = 20 * np.log10(avg_mags * amp_factor * rms_factor / mic_factor)
    
    
    lowess = sm.nonparametric.lowess
    # === deal w/ loess
    sigma= 0.08  # local-ness factor {0.1-0.2}
    fit= lowess(noise_floor,f/1000,frac=sigma, return_sorted=False)
    
    plt.plot(f, noise_floor)
    plt.plot(f, fit,'g--',label='loess fit')
    plt.title(fp.name)
    plt.show()
    


